Recreates the counter intuitive example from this article, gets the same results

https://www.linkedin.com/feed/update/urn:li:activity:7262888835678687232/?commentUrn=urn%3Ali%3Acomment%3A(ugcPost%3A7262888832465846272%2C7263912720486330368)&dashCommentUrn=urn%3Ali%3Afsd_comment%3A(7263912720486330368%2Curn%3Ali%3AugcPost%3A7262888832465846272)&dashReplyUrn=urn%3Ali%3Afsd_comment%3A(7264642840423452672%2Curn%3Ali%3AugcPost%3A7262888832465846272)&replyUrn=urn%3Ali%3Acomment%3A(ugcPost%3A7262888832465846272%2C7264642840423452672)

In [1]:
include(dirname(dirname(pwd()))*"\\src\\TuLiPa.jl");
using .TuLiPa
using Dates, DataFrames, CSV, JSON, Plots, JuMP, HiGHS
include(dirname(dirname(pwd())) * raw"\demos\other\flowbased_utils.jl")

remove_cap_for_flowbased_transm (generic function with 1 method)

In [2]:
PowerA = 100.0
PowerA_cap = 1000.0 / 0.024
DemandA = 140.0 / 0.024
PowerB_cap = 1000.0 / 0.024
PowerB = 200.0
DemandB = 170.0 / 0.024
PowerC_cap = 20.0  / 0.024
PowerC = 70.0
DemandC = 140.0 / 0.024

A = "A" #"PowerBalance_A"
B = "B" #"PowerBalance_B"
C = "C" #"PowerBalance_C"


function make_connection(from, to, prefix = "")  
    
    transm_from = replace(from, ("PowerBalance_" => ""))
    transm_to = replace(to, ("PowerBalance_" => ""))
    
    flow_name = "Transm_$(transm_from)->$(transm_to)" * prefix
    arrow_name_from = "arrow_from$(from)$(to)" * prefix
    arrow_name_to = "arrow_to$(from)$(to)" * prefix
    balance_from = "$(from)"
    balance_to = "$(to)"
    cap_name = "$(flow_name)_cap"
    elem = [
        DataElement("Flow", "BaseFlow", flow_name, Dict()),    
        DataElement("Arrow", "BaseArrow", arrow_name_from, Dict("Balance" => balance_from, "Flow" => flow_name, "Conversion" => 1.0, "Direction" => "Out")),
        DataElement("Arrow", "BaseArrow", arrow_name_to, Dict("Balance" => balance_to, "Flow" => flow_name, "Conversion" => 1.0, "Direction" => "In")),
    ]    
    return elem
end

make_connection (generic function with 2 methods)

In [3]:
function create_elements()
    elements = [

        DataElement("Balance", "BaseBalance", A, Dict{Any, Any}("Commodity" => "Power")),    
        DataElement("Flow", "BaseFlow", "PowerA", Dict{Any, Any}()),
        DataElement("Arrow", "BaseArrow", "ArrowA", Dict{Any, Any}("Balance" => A, "Flow" => "PowerA", "Conversion" => 1.0, "Direction" => "In")),
        DataElement("Cost", "CostTerm", "PowerA_cost", Dict{Any, Any}("Param" => PowerA, "WhichInstance" => "PowerA", "WhichConcept" => "Flow", "Direction" => "In")),
        DataElement("Capacity", "PositiveCapacity", "PowerA_cap", Dict{Any, Any}("Param" => "PowerA_cap", "WhichInstance" => "PowerA", "WhichConcept" => "Flow", "Bound" => "Upper")),
        DataElement("Param", "MWToGWhSeriesParam", "PowerA_cap", Dict{Any, Any}("Level" => PowerA_cap, "Profile" => 1.0)),
        DataElement("Param", "MWToGWhSeriesParam", "DemandA", Dict("Level" => DemandA , "Profile" => 1.0)),
        DataElement("RHSTerm", "BaseRHSTerm", "DemandA", Dict{Any, Any}("Balance" => A, "Param" => "DemandA", "Direction" => "Out")),

        DataElement("Balance", "BaseBalance", B, Dict{Any, Any}("Commodity" => "Power")),    
        DataElement("Flow", "BaseFlow", "PowerB", Dict{Any, Any}()),
        DataElement("Arrow", "BaseArrow", "ArrowB", Dict{Any, Any}("Balance" => B, "Flow" => "PowerB", "Conversion" => 1.0, "Direction" => "In")),
        DataElement("Cost", "CostTerm", "PowerB_cost", Dict{Any, Any}("Param" => PowerB, "WhichInstance" => "PowerB", "WhichConcept" => "Flow", "Direction" => "In")),
        DataElement("Capacity", "PositiveCapacity", "PowerB_cap", Dict{Any, Any}("Param" => "PowerB_cap", "WhichInstance" => "PowerB", "WhichConcept" => "Flow", "Bound" => "Upper")),
        DataElement("Param", "MWToGWhSeriesParam", "PowerB_cap", Dict{Any, Any}("Level" => PowerB_cap, "Profile" => 1.0)),
        DataElement("Param", "MWToGWhSeriesParam", "DemandB", Dict("Level" => DemandB , "Profile" => 1.0)),
        DataElement("RHSTerm", "BaseRHSTerm", "DemandB", Dict{Any, Any}("Balance" => B, "Param" => "DemandB", "Direction" => "Out")),

        DataElement("Balance", "BaseBalance", C, Dict{Any, Any}("Commodity" => "Power")),    
        DataElement("Flow", "BaseFlow", "PowerC", Dict{Any, Any}()),
        DataElement("Arrow", "BaseArrow", "ArrowC", Dict{Any, Any}("Balance" => C, "Flow" => "PowerC", "Conversion" => 1.0, "Direction" => "In")),
        DataElement("Cost", "CostTerm", "PowerC_cost", Dict{Any, Any}("Param" => PowerC, "WhichInstance" => "PowerC", "WhichConcept" => "Flow", "Direction" => "In")),
        DataElement("Capacity", "PositiveCapacity", "PowerC_cap", Dict{Any, Any}("Param" => "PowerC_cap", "WhichInstance" => "PowerC", "WhichConcept" => "Flow", "Bound" => "Upper")),
        DataElement("Param", "MWToGWhSeriesParam", "PowerC_cap", Dict{Any, Any}("Level" => PowerC_cap, "Profile" => 1.0)),
        DataElement("Param", "MWToGWhSeriesParam", "DemandC", Dict("Level" => DemandC , "Profile" => 1.0)),
        DataElement("RHSTerm", "BaseRHSTerm", "DemandC", Dict{Any, Any}("Balance" => C, "Param" => "DemandC", "Direction" => "Out"))
    ]

    elements = vcat(elements, make_connection(A, B))
    elements = vcat(elements, make_connection(B, A))

    elements = vcat(elements, make_connection(A, C))
    elements = vcat(elements, make_connection(C, A))

    elements = vcat(elements, make_connection(B, C))
    elements = vcat(elements, make_connection(C, B))

    power_horizon = SequentialHorizon(1, Day(1))
    push!(elements, getelement(COMMODITY_CONCEPT, "BaseCommodity", "Power", (HORIZON_CONCEPT, power_horizon)))
    addscenariotimeperiod!(elements, "ScenarioTimePeriod", getisoyearstart(1981), getisoyearstart(1983));
    return elements
end

create_elements (generic function with 1 method)

In [4]:

ptdfs = DataFrame(
    line=["Transm_A->B", "Transm_A->C", "Transm_B->C"],
    RAM = [100000.0, 10.0, 100000.0],
    A = [0.91, 0.09, -0.09],
    C = [0.09, -0.09, -0.91],
    B = [0, 0, 0]
)





df = ptdfs

df_copy = copy(df)
df_copy.RAM .= missing
df_copy.border .= true
df_copy
df.border .= false
df.line = "non_flow" .* df.line
df = vcat(df, df_copy)
select(df, ["line", "RAM", "border"], Not(["line", "RAM", "border"]))

Row,line,RAM,border,A,C,B
,String,Float64?,Bool,Float64,Float64,Int64
1,non_flowTransm_A->B,100000.0,false,0.91,0.09,0
2,non_flowTransm_A->C,10.0,false,0.09,-0.09,0
3,non_flowTransm_B->C,100000.0,false,-0.09,-0.91,0
4,Transm_A->B,missing,true,0.91,0.09,0
5,Transm_A->C,missing,true,0.09,-0.09,0
6,Transm_B->C,missing,true,-0.09,-0.91,0


# With flowbased

In [7]:
elements = create_elements() 
transm = [e for e in elements if is_transmission(e)]
T = DataElement
flow_based = make_flowbased(df, transm)
flow_based = Vector{DataElement}(flow_based)
elements = Vector{DataElement}(elements)

elements = vcat(elements,  flow_based)
modelobjects = getmodelobjects(elements)
mymodel = JuMP.Model(HiGHS.Optimizer)
prob = JuMP_Prob(modelobjects, mymodel)

datatime = getisoyearstart(2023)
scenariotime = getisoyearstart(1981)
prob_time = TwoTime(datatime, scenariotime)    
update!(prob, prob_time)
solve!(prob)
print(prob.model)
print(solution_summary(prob.model, verbose = true))
    
-dual.(prob.model[:BalanceA]), -dual.(prob.model[:BalanceB]), -dual.(prob.model[:BalanceC])


* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 7.42889e+04
  Objective bound    : 0.00000e+00
  Relative gap       : Inf
  Dual objective value : 7.42889e+04
  Primal solution :
    FlowPowerA[1] : 1.31111e+02
    FlowPowerB[1] : 2.98889e+02
    FlowPowerC[1] : 2.00000e+01
    FlowTransm_A->B[1] : 0.00000e+00
    FlowTransm_A->C[1] : 1.00000e+01
    FlowTransm_B->A[1] : 1.88889e+01
    FlowTransm_B->C[1] : 1.10000e+02
    FlowTransm_C->A[1] : 0.00000e+00
    FlowTransm_C->B[1] : 0.00000e+00
  Dual solution :
    BalanceA[1] : -1.00000e+02
    BalanceB[1] : -2.00000e+02
    BalanceC[1] : -3.00000e+02
    FlowBasedTransm_A->B[1] : -0.00000e+00
    FlowBasedTransm_A->C[1] : -0.00000e+00
    FlowBasedTransm_B->C[1] : -0.00000e+00
    FlowBasednon_flowTransm_A->Bin[1] :

([100.0], [200.0], [300.0])

# without flowbased

In [8]:
elements = create_elements()
transm = [e for e in elements if is_transmission(e)]
T = DataElement

elements = Vector{DataElement}(elements)

modelobjects = getmodelobjects(elements)
mymodel = JuMP.Model(HiGHS.Optimizer)
prob = JuMP_Prob(modelobjects, mymodel)
datatime = getisoyearstart(2023)
scenariotime = getisoyearstart(1981)
prob_time = TwoTime(datatime, scenariotime)    
update!(prob, prob_time)
solve!(prob)
print(prob.model)
print(solution_summary(prob.model, verbose = true))
    
-dual.(prob.model[:BalanceA]), -dual.(prob.model[:BalanceB]), -dual.(prob.model[:BalanceC])


* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 4.44000e+04
  Objective bound    : 0.00000e+00
  Relative gap       : Inf
  Dual objective value : 4.44000e+04
  Primal solution :
    FlowPowerA[1] : 4.30000e+02
    FlowPowerB[1] : 0.00000e+00
    FlowPowerC[1] : 2.00000e+01
    FlowTransm_A->B[1] : 1.70000e+02
    FlowTransm_A->C[1] : 1.20000e+02
    FlowTransm_B->A[1] : 0.00000e+00
    FlowTransm_B->C[1] : 0.00000e+00
    FlowTransm_C->A[1] : 0.00000e+00
    FlowTransm_C->B[1] : 0.00000e+00
  Dual solution :
    BalanceA[1] : -1.00000e+02
    BalanceB[1] : -1.00000e+02
    BalanceC[1] : -1.00000e+02

* Work counters
  Solve time (sec)   : 9.94921e-04
  Simplex iterations : 3
  Barrier iterations : 0
  Node count         : -1


([100.0], [100.0], [100.0])